[adaptado de [Programa de cursos integrados Aprendizado de máquina](https://www.coursera.org/specializations/machine-learning-introduction) de [Andrew Ng](https://www.coursera.org/instructor/andrewng)  ([Stanford University](http://online.stanford.edu/), [DeepLearning.AI](https://www.deeplearning.ai/) ) ]

In [ ]:
# Baixar arquivos adicionais para o laboratório.
!wget https://github.com/fabiobento/dnn-course-2024-1/raw/main/00_course_folder/ml_intro/class_03/Laborat%C3%B3rios/lab_utils_ml_intro_week_3.zip
!unzip -n -q lab_utils_ml_intro_week_3.zip

In [ ]:
# Testar se estamos no Google Colab
try:
  import google.colab
  IN_COLAB = True
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  IN_COLAB = False

# Gradiente Descendente para Regressão Logística

## Objetivos
Neste laboratório, você irá:
- atualizar a descida do gradiente para regressão logística.
- explorar a descida gradiente em um conjunto de dados familiar

In [ ]:
!pip install ipympl

In [ ]:
import copy, math
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
from lab_utils_common import  dlc, plot_data, plt_tumor_data, sigmoid, compute_cost_logistic
from plt_quad_logistic import plt_quad_logistic, plt_prob
plt.style.use('./deeplearning.mplstyle')

## Conjunto de dados
Vamos começar com os mesmos dois conjuntos de dados de recursos usados no laboratório de fronteiras de decisão.

In [ ]:
X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_train = np.array([0, 0, 0, 1, 1, 1])

Como antes, usaremos uma função auxiliar para representar graficamente esses dados. Os pontos de dados com rótulo $y=1$ são mostrados como cruzes vermelhas, enquanto os pontos de dados com rótulo $y=0$ são mostrados como círculos azuis.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,4))
plot_data(X_train, y_train, ax)

ax.axis([0, 4, 0, 3.5])
ax.set_ylabel('$x_1$', fontsize=12)
ax.set_xlabel('$x_0$', fontsize=12)
plt.show()

## Gradiente Descendente Logístico
<img align="right" src="./images/C1_W3_Logistic_gradient_descent.png"     style=" width:400px; padding: 10px; " >

Lembre-se de que o algoritmo de descida gradiente utiliza o cálculo de gradiente:
$$\begin{align*}
&\text{repetir até a convergência:} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$

Onde cada iteração realiza atualizações simultâneas em $w_j$ para todos os $j$, onde
$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3} 
\end{align*}$$

* m é o número de exemplos de treinamento no conjunto de dados
* $f_{\mathbf{w},b}(x^{(i)})$ é a previsão do modelo, enquanto $y^{(i)}$ é o alvo
* Para um modelo de regressão logística
    $z = \mathbf{w} \cdot \mathbf{x} + b$  
    $f_{\mathbf{w},b}(x) = g(z)$  
    onde $g(z)$ é a função sigmóide:
    $g(z) = \frac{1}{1+e^{-z}}$   
    


### Implementação de gradiente descendente
A implementação do algoritmo de descida gradiente tem dois componentes:
- A equação de implementação do loop (1) acima. Isso é `gradient_descent` abaixo e geralmente é fornecido a você em laboratórios práticos anteriores.
- O cálculo do gradiente atual, equações (2,3) acima. Isto é `compute_gradient_logistic` abaixo. Você será solicitado a implementar o laboratório prático desta semana.

#### Calculando o gradiente, descrição do código
Implementa a equação (2),(3) acima para todos $w_j$ e $b$.
Existem muitas maneiras de implementar isso. Descrito abaixo é o seguinte:
- inicializar variáveis para acumular `dj_dw` e `dj_db`
- para cada exemplo
     - calcule o erro para esse exemplo $g(\mathbf{w} \cdot \mathbf{x}^{(i)} + b) - \mathbf{y}^{(i)}$
     - para cada valor de entrada $x_{j}^{(i)}$ neste exemplo,
         - multiplique o erro pela entrada $x_{j}^{(i)}$ e adicione ao elemento correspondente de `dj_dw`. (equação 2 acima)
     - adicione o erro ao `dj_db` (equação 3 acima)

- divida `dj_db` e `dj_dw` pelo número total de exemplos (m)
- observe que $\mathbf{x}^{(i)}$ em numpy `X[i,:]` ou `X[i]` e $x_{j}^{(i)}$ é `X[ eu, j]`

In [ ]:
def compute_gradient_logistic(X, y, w, b): 
    """
    Calcula o gradiente para regressão logística
 
    Args:
      X (ndarray (m,n): DAdos, m exemplos com b recursos
      y (ndarray (m,)): valores alvo
      w (ndarray (n,)): parâmetros do modelo  
      b (scalar)      : parâmetros do modelo
    Returns
      dj_dw (ndarray (n,)): O gradiente do custo em relação ao valor os parâmetros w.
      dj_db (scalar)      : O gradiente do custo em relação ao valor os parâmetros b.
    """
    m,n = X.shape
    dj_dw = np.zeros((n,))                           #(n,)
    dj_db = 0.

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i],w) + b)          #(n,)(n,)=scalar
        err_i  = f_wb_i  - y[i]                       #escalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]      #escalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                   #(n,)
    dj_db = dj_db/m                                   #escalar
        
    return dj_db, dj_dw  

Verifique a implementação da função gradiente usando a célula abaixo.

In [ ]:
X_tmp = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
y_tmp = np.array([0, 0, 0, 1, 1, 1])
w_tmp = np.array([2.,3.])
b_tmp = 1.
dj_db_tmp, dj_dw_tmp = compute_gradient_logistic(X_tmp, y_tmp, w_tmp, b_tmp)
print(f"dj_db: {dj_db_tmp}" )
print(f"dj_dw: {dj_dw_tmp.tolist()}" )

**Saída Esperada**
``` 
dj_db: 0.49861806546328574
dj_dw: [0.498333393278696, 0.49883942983996693]
```

#### Código de gradiente descendente
A equação de implementação do código (1) acima é implementada abaixo. Reserve um momento para localizar e comparar as funções da rotina com as equações acima.

In [ ]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters): 
    """
    Executa descida gradiente em lote
    
    Args:
      X (ndarray (m,n)   : Dados, m exemplos com n recursos
      y (ndarray (m,))   : valores alvo
      w_in (ndarray (n,)): Valores iniciais dos parâmetros do modelo
      b_in (scalar)      : Valores iniciais dos parâmetros do modelo
      alpha (float)      : Taxa de aprendizado
      num_iters (scalar) : Número de iterações para executar a descida gradiente
      
    Returns:
      w (ndarray (n,))   : Valores atualizados dos parâmetros
      b (scalar)         : Valores atualizados dos parâmetros
    """
    # Uma matriz para armazenar os custos J e w em cada iteração, principalmente para gráficos posteriores
    J_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in
    
    for i in range(num_iters):
        # Calcule o gradiente e atualize os parâmetros
        dj_db, dj_dw = compute_gradient_logistic(X, y, w, b)   

        # Atualizar parâmetros usando w, b, alfa e gradiente
        w = w - alpha * dj_dw               
        b = b - alpha * dj_db               
      
        # Salve o custo J em cada iteração
        if i<100000:      # evitar exaustão de recursos 
            J_history.append( compute_cost_logistic(X, y, w, b) )

        # Imprima o custo a cada intervalo de 10 vezes ou tantas iterações se <10
        if i% math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]}   ")
        
    return w, b, J_history #retorna o histórico final de w,b e J para gráficos


Vamos executar a descida gradiente em nosso conjunto de dados.

In [ ]:
w_tmp  = np.zeros_like(X_train[0])
b_tmp  = 0.
alph = 0.1
iters = 10000

w_out, b_out, _ = gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters) 
print(f"\nparâmetros atualizados: w:{w_out}, b:{b_out}")

#### Vamos traçar os resultados da descida do gradiente:

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,4))
# plotar a probabilidade
plt_prob(ax, w_out, b_out)

# Plotar os dados originais
ax.set_ylabel(r'$x_1$')
ax.set_xlabel(r'$x_0$')   
ax.axis([0, 4, 0, 3.5])
plot_data(X_train,y_train,ax)

# Plotar a fronteira de decisão
x0 = -b_out/w_out[0]
x1 = -b_out/w_out[1]
ax.plot([0,x0],[x1,0], c=dlc["dlblue"], lw=1)
plt.show()

No gráfico acima:
  - o sombreamento reflete a probabilidade y=1 (resultado antes da fronteira de decisão)
  - a fronteira de decisão é a linha na qual a probabilidade = 0,5

## Outro conjunto de dados
Voltemos a um conjunto de dados de uma variável. Com apenas dois parâmetros, $w$, $b$, é possível traçar a função de custo usando um gráfico de contorno para ter uma ideia melhor do que está acontecendo a descida do gradiente.

In [ ]:
x_train = np.array([0., 1, 2, 3, 4, 5])
y_train = np.array([0,  0, 0, 1, 1, 1])

Como antes, usaremos uma função auxiliar para representar graficamente esses dados. Os pontos de dados com rótulo $y=1$ são mostrados como cruzes vermelhas, enquanto os pontos de dados com rótulo $y=0$ são mostrados como círculos azuis.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(4,3))
plt_tumor_data(x_train, y_train, ax)
plt.show()

No gráfico abaixo, tente:
- alterando $w$ e $b$ clicando no gráfico de contorno no canto superior direito.
     - as alterações podem levar um ou dois segundos
     - observe a variação do valor do custo no gráfico superior esquerdo.
     - observe que o custo é acumulado por uma perda em cada exemplo (linhas pontilhadas verticais)
- execute a descida gradiente clicando no botão laranja.
     - observe o custo cada vez menor (contorno e gráfico de custo estão em log (custo)
     - clicar no gráfico de contorno redefinirá o modelo para uma nova execução
- para redefinir o gráfico, execute novamente a célula

In [ ]:
w_range = np.array([-1, 7])
b_range = np.array([1, -14])
quad = plt_quad_logistic( x_train, y_train, w_range, b_range )

## Parabéns!
Você:
- examinou as fórmulas e implementação do cálculo do gradiente para regressão logística
- utilizou essas rotinas em
     - explorar um único conjunto de dados variáveis
     - explorar um conjunto de dados de duas variáveis